# Import Libraries

In [1]:
import pandas as pd
import re
import string
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Dataset

In [4]:
df = pd.read_csv('tweets.csv')

In [5]:
df.shape

(25798, 12)

In [6]:
df.sample(3)

,id,id_str,created_at,text,name,screen_name,location,followers_count,geo,place,retweet_count,favorite_count
23039,1.516156e+18,1516155893410316298,Mon Apr 18 20:45:08 +0000 2022,RT @yoshkinkrot: ❗️ Христо Грозев: «Сообщалось...,Djemal ДжемАльт 🇷🇺🇺🇦🇫🇷,Djemalozieux,NaN,356.0,NaN,NoData,23.0,0.0
4147,1.514370e+18,1514370263705653256,Wed Apr 13 22:29:41 +0000 2022,RT @Vcex_naxui: 2 июня 2014 года. Украина. Луг...,Евгений Кустов,eugene_kust,Россия,549.0,NaN,NoData,148.0,0.0
11645,1.516566e+18,1516565503942275077,Tue Apr 19 23:52:47 +0000 2022,RT @antiputler_news: Россия развернула ЗРК С-4...,Валерiй,i01435933,Україна,1071.0,NaN,NoData,363.0,0.0


# Pre-processing

## Cleaning up tweets

In [7]:
df = df[df['text'].notna()]

Remove all mentions of users (@***)

In [8]:
df['text_clean'] = [re.sub(r'@\w+', r'', x) for x in df['text']]

Remove line breaks

In [9]:
df['text_clean'] = [x.replace('\n', '') for x in df['text_clean']]

Convert to lowercase

In [10]:
df['text_clean'] = [x.lower() for x in df['text_clean']]

Remove mentions rt

In [11]:
df['text_clean'] = [re.sub(r'rt :', r'', x) for x in df['text_clean']]
df['text_clean'] = [re.sub(r'rt', r'', x) for x in df['text_clean']]

Remove all links

In [12]:
df['text_clean'] = [re.sub(r'http\S+', r'', x) for x in df['text_clean']]

Remove emoji

In [13]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [14]:
df['text_clean'] = [emoji_pattern.sub(r'', x) for x in df['text_clean']]

In [15]:
df['text_clean'] = [re.sub(r'❗️', r'', x) for x in df['text_clean']]
df['text_clean'] = [re.sub(r'⚔️', r'', x) for x in df['text_clean']]
df['text_clean'] = [re.sub(r'⚡', r'', x) for x in df['text_clean']]

Remove extra spaces

In [16]:
df['text_clean'] = [x.strip() for x in df['text_clean']]

Remove partial punctuation

In [17]:
df['text_clean'] = [re.sub(r'"', r'', x) for x in df['text_clean']]
df['text_clean'] = [re.sub(r'«', r'', x) for x in df['text_clean']]
df['text_clean'] = [re.sub(r'»', r'', x) for x in df['text_clean']]
df['text_clean'] = [re.sub(r'!', r'', x) for x in df['text_clean']]
#df['text_clean'] = [re.sub(r'...', r'', x) for x in df['text_clean']]

Remove numbers

In [18]:
df['text_clean'] = [re.sub('\d+', '', x) for x in df['text_clean']]

## What we have after cleaning

In [19]:
df[['text', 'text_clean']].sample(10)

,text,text_clean
4653,@ds107m6p @ron_ov @dw_russian Мне хорошо извес...,"мне хорошо известно, что татары крымские и тот..."
24847,RT @antiputler_news: 😩 Вот такой сегодня был п...,вот такой сегодня был прилет по гаражным коопе...
22083,RT @EvaBorisovna: Я лично согласна отказаться ...,"я лично согласна отказаться от русской музыки,..."
5924,"RT @Grantoedov: ⚡️Новые трофеи из ПОПАСНОЙ, уж...","️новые трофеи из попасной, уже у казаков на во..."
11281,На войне погибли до 3000 украинских солдат. По...,на войне погибли до украинских солдат. потери...
6347,RT @wargonzoo: ⚡️Авдеевский котёл⚡️Обстановка ...,️авдеевский котёл️обстановка на вечер ..️после...
17231,RT @leonidvolkov: Не существует ни одного пере...,не существует ни одного перевода ни на один ру...
4548,RT @euro2012uaorg: Видео из Рубежного Луганско...,видео из рубежного луганской области. тяжелая ...
14795,RT @dedzaebal: Лавров : Россия не собирается м...,лавров : россия не собирается менять режим в у...
14533,RT @Geschichter: Кремль: Если Украина не сдаст...,"кремль: если украина не сдастся, мы ядерную бо..."


## Tokenization

In [20]:
russian_stop_words = stopwords.words("russian")
snowball = SnowballStemmer(language='russian')


def tokenize_text(x):
    tokens = word_tokenize(x, language='russian')
    tokens_no_punkt = [i for i in tokens if i not in string.punctuation]
    tokens_no_stopwords = [i for i in tokens_no_punkt if i not in russian_stop_words]
    stemmed_tokens = [snowball.stem(i) for i in tokens_no_stopwords]
    return stemmed_tokens

In [21]:
df['text_tokenized'] = [tokenize_text(x) for x in df['text_clean']]

In [22]:
df[['text', 'text_clean', 'text_tokenized']].sample(15)

,text,text_clean,text_tokenized
12025,"ВСУ уничтожили одного из командиров потешной ""...",всу уничтожили одного из командиров потешной л...,"[всу, уничтож, одн, командир, потешн, лнр, миш..."
12072,RT @chuuyamyboy: тред!! персонажи бсд как укра...,тред персонажи бсд как украинские подростки во...,"[тред, персонаж, бсд, украинск, подростк, врем..."
6374,RT @rianru: За последние сутки в ДНР погибли т...,за последние сутки в днр погибли трое военносл...,"[последн, сутк, днр, погибл, тро, военнослужа,..."
7075,RT @altangerelch1: АНУ Украйнд донбас болон зү...,ану украйнд донбас болон зүүн украйныг чөлөөлө...,"[ан, украйнд, донбас, болон, зүүн, украйныг, ч..."
4399,RT @GirkinGirkin: Луганск https://t.co/Ihku9y8UH3,луганск,[луганск]
6730,"RT @DonbassSegodnya: Мариуполь, на трассе стои...","мариуполь, на трассе стоит престарелая женщина...","[мариупол, трасс, сто, престарел, женщин, свеч..."
11151,@abunin Зеленский в панике 👀,зеленский в панике,"[зеленск, паник]"
8872,"RT @eskovoroda: Написали, как люди покидали Ма...","написали, как люди покидали мариуполь — одно и...","[написа, люд, покида, мариупол, —, одн, сам, с..."
21066,RT @RozovayaShayrma: да блять эта хуйня происх...,да блять эта хуйня происходит годамимудло отру...,"[блят, эт, хуйн, происход, годамимудл, отруба,..."
6110,RT @Alla91748059: Россия теряет поддержку Каза...,россия теряет поддержку казахстана после начал...,"[росс, теря, поддержк, казахста, нача, войн, у..."


# Toxic Model

Russian Language Toxic Comments https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments

In [23]:
df_train = pd.read_csv('labeled.csv')

In [24]:
df_train

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [25]:
df_train['toxic'] = df_train['toxic'].astype(int)

In [26]:
df_train['comment_clear'] = df_train['comment'].str.lower()
df_train['comment_clear'] = [re.sub('\d+', '', x) for x in df_train['comment_clear']]

In [27]:
df_train['comment_tokenized'] = [tokenize_text(x) for x in df_train['comment_clear']]

In [28]:
pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = lambda x: tokenize_text(x))),
    ("model", RandomForestClassifier(n_jobs=-1))
])

In [29]:
pipeline.fit(df_train['comment_clear'], df_train['toxic'])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7fcf485c9af0>)),
                ('model', RandomForestClassifier(n_jobs=-1))])

## Apply on our tweets

In [31]:
df['toxic_proba'] = list(pipeline.predict_proba(df['text_clean']))
df['toxic'] = list(pipeline.predict(df['text_clean']))

In [33]:
df[['text', 'text_clean', 'toxic_proba', 'toxic']].sample(10)

,text,text_clean,toxic_proba,toxic
25299,"RT @sashatimofeevax: ""Зачем нам насиловать укр...",зачем нам насиловать украинок?- возмутились ру...,"[0.4, 0.6]",1
8448,RT @i_army_org: Последствия вступления в НАТО ...,последствия вступления в нато очевидны - это б...,"[0.84, 0.16]",0
3616,RT @rianru: Киев хочет обратиться к Москве с т...,киев хочет обратиться к москве с требованием в...,"[0.8, 0.2]",0
14869,RT @KattieBanned: @prostotak182 @hUDhfnQFzM65l...,"то, что спецоперация по захвату власти в украи...","[0.48, 0.52]",1
5844,"@christogrozev Знаете, у кого ТОЧНО ОФИЦИАЛЬНО...","знаете, у кого точно официально % есть точка-у...","[0.88, 0.12]",0
20694,RT @ttt_mir_no: @stranabolna предлагает забаст...,предлагает забастовку в виде синхронных больни...,"[0.96, 0.04]",0
8535,RT @globusnewsorg: Премьер-министр Финляндии з...,премьер-министр финляндии заявила о вступлении...,"[0.94, 0.06]",0
12803,RT @nastyabakulina_: Этой империи нужен и Киев...,"этой империи нужен и киев, и крым.может быть я...","[0.7, 0.3]",0
19178,RT @KermlinRussia: В конце апреля агрессивный ...,в конце апреля агрессивный блок нато планирует...,"[0.6825, 0.3175]",0
23972,RT @anders_aslund: в ожидании битвы за Донбасс...,в ожидании битвы за донбасс борьба в кремле пр...,"[0.4, 0.6]",1
